# Quickstart to Langchain with Google Gemini
In this notebook, we'll look to get up & running with LangChain and Google Gemini.

To use Gemini API you'll need to sign up for the API access, which is still free by the way (at least as of June '24) - Thanks Google!, and acquire an API key. For instructions [see this link](). Generate a new API token and save it in a local `.env` file as
```
OPENAI_API_KEY=sk-XXXXXX
````
Replace `sk-XXXXXX` with your API key. **Don't share this with ANYONE**. Add `.env` to the `.gitignore` file, so that this file is never uploaded to GitHub (assuming you are using Git for version control).

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

# load API keys from local .env file
load_dotenv(find_dotenv())

True

In [4]:
# instantiate the LLM instance
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    # we will be using just the text model
    model="gemini-pro",
    google_api_key=os.environ["GOOGLE_API_KEY"],
    # No creativity whatsoever!
    temperature=0.0,
    # LangChain hack as Gemini does not support system prompts
    # as a separate category, though we can set the context using
    # a human prompt as a system prompt
    convert_system_message_to_human=True,
)

In [5]:
# modify the question below & try
question = """
What is Deepika Padukone's birthday? Which country won the cricket world cup the year she was born?
"""
response = llm.invoke(question)
print(response.content)

/home/mjbhobe/code/git-projects/LangChain/myenv/lib/python3.9/site-packages/langchain_google_genai/chat_models.py:352: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


* Deepika Padukone's birthday: **January 5, 1986**
* Country that won the cricket world cup in 1986: **Australia**


In [6]:
# let's us a prompt template instead
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        # here is where the convert_system_message_to_human=True,
        # parameter to the ChatGoogleGenerativeAI() constructor
        # becomes relevant
        ("system", "You are a world class technical documentation writer."),
        ("user", "{input}"),
    ]
)

In [7]:
chain = prompt | llm
response = chain.invoke({"input": "How can RAG help LLMs with Q&A?"})
print(response.content)

/home/mjbhobe/code/git-projects/LangChain/myenv/lib/python3.9/site-packages/langchain_google_genai/chat_models.py:352: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


**RAG (Retrieval-Augmented Generation)** can significantly enhance the Q&A capabilities of Large Language Models (LLMs) by leveraging external knowledge sources. Here's how RAG can assist LLMs with Q&A:

**1. Knowledge Retrieval:**
RAG retrieves relevant information from external knowledge bases, such as Wikipedia, news articles, or domain-specific databases. This allows LLMs to access a broader range of information beyond their own training data.

**2. Contextualization:**
RAG provides LLMs with the context of the user's question by retrieving related documents. This helps LLMs understand the intent and specific requirements of the query.

**3. Answer Generation:**
LLMs use the retrieved knowledge to generate comprehensive and accurate answers. RAG ensures that the answers are grounded in factual information and relevant to the user's question.

**4. Answer Refinement:**
RAG can further refine the answers generated by LLMs by identifying and correcting potential errors or inconsistenc

Notice that the output above has lots of markdown. We can display the markdown in the output cell of a notebook using the following code.

In [8]:
import textwrap
from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
    text = text.replace("•", "  *")
    return Markdown(textwrap.indent(text, "> ", predicate=lambda _: True))


# instead of print(response.content), use
to_markdown(response.content)

> **RAG (Retrieval-Augmented Generation)** can significantly enhance the Q&A capabilities of Large Language Models (LLMs) by leveraging external knowledge sources. Here's how RAG can assist LLMs with Q&A:
> 
> **1. Knowledge Retrieval:**
> RAG retrieves relevant information from external knowledge bases, such as Wikipedia, news articles, or domain-specific databases. This allows LLMs to access a broader range of information beyond their own training data.
> 
> **2. Contextualization:**
> RAG provides LLMs with the context of the user's question by retrieving related documents. This helps LLMs understand the intent and specific requirements of the query.
> 
> **3. Answer Generation:**
> LLMs use the retrieved knowledge to generate comprehensive and accurate answers. RAG ensures that the answers are grounded in factual information and relevant to the user's question.
> 
> **4. Answer Refinement:**
> RAG can further refine the answers generated by LLMs by identifying and correcting potential errors or inconsistencies. This improves the quality and reliability of the Q&A output.
> 
> **5. Knowledge Update:**
> RAG enables LLMs to continuously update their knowledge base by incorporating new information from external sources. This ensures that the Q&A system remains up-to-date and provides the most relevant answers.
> 
> **Benefits of RAG for LLM Q&A:**
> 
> * **Improved Accuracy:** RAG provides LLMs with access to factual information, reducing the risk of incorrect or biased answers.
> * **Enhanced Contextualization:** RAG helps LLMs understand the context of the question, leading to more relevant and specific answers.
> * **Broader Knowledge Base:** RAG expands the knowledge base of LLMs, allowing them to answer a wider range of questions.
> * **Continuous Learning:** RAG enables LLMs to continuously update their knowledge, ensuring the Q&A system remains current.
> * **Reduced Bias:** By leveraging external knowledge sources, RAG helps mitigate potential biases in LLM training data.
> 
> In summary, RAG plays a crucial role in enhancing the Q&A capabilities of LLMs by providing them with relevant knowledge, contextualization, and answer refinement. This results in more accurate, comprehensive, and up-to-date answers, making LLMs more effective for Q&A tasks.

Ah! That's much better. Now let's see what effect adding an output parser, such as `StrOutputParser()` has on the output end of the chain.

In [9]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [10]:
# here we string them together - prompt -> llm -> output parser
chain2 = prompt | llm | output_parser

In [11]:
# and voila
response = chain.invoke({"input": "How can RAG help LLMs with Q&A?"})
print(response.content)

/home/mjbhobe/code/git-projects/LangChain/myenv/lib/python3.9/site-packages/langchain_google_genai/chat_models.py:352: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


**RAG (Retrieval-Augmented Generation)** can significantly enhance the Q&A capabilities of Large Language Models (LLMs) by leveraging external knowledge sources. Here's how RAG can assist LLMs with Q&A:

**1. Knowledge Retrieval:**
RAG retrieves relevant information from a large knowledge base, such as Wikipedia or a domain-specific corpus. This knowledge is used to augment the LLM's internal knowledge, providing it with a broader context and deeper understanding.

**2. Contextualization:**
RAG helps LLMs understand the context of the user's question by providing relevant passages or documents. This contextualization enables the LLM to generate more accurate and comprehensive answers.

**3. Fact Verification:**
RAG can verify the accuracy of the LLM's answers by comparing them to the retrieved knowledge. This ensures that the answers are factually correct and reliable.

**4. Answer Generation:**
RAG can generate answers by combining the retrieved knowledge with the LLM's own language g

Now let us query a website for the ICC Cricket World Cup 1987, which was co-hosted by India & Pakistan, and ask some questions. By default, our LLM _may not_ have this knowledge. So we will help it by adding additional context to our query using RAG.
In this process, we'll look up relevant documents/information using a _Retriever_ class, which is then passed to our LLM as additional context. We'll populate a vector store (local storage) and use that as a retriever.

Please ensure that you have installed `beautifulsoup4` to help parse data from a webpage.
```bash
$> pip install beautifulsoup4
```

In [13]:
from langchain_community.document_loaders import WebBaseLoader

# load the documents - this code fragment just loads the documents. It
# does not vectorize anything
world_cup_url = "https://en.wikipedia.org/wiki/1987_Cricket_World_Cup"
loader = WebBaseLoader(world_cup_url)

docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [14]:
# here is the code to embed the web page into a local vector store
import os, pathlib
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

# create directory to save vector store
save_path = pathlib.Path(os.getcwd()) / "vecstore"
save_path.mkdir(parents=True, exist_ok=True)
vec_store_path = save_path / "faiss_index_gemini"

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

if vec_store_path.exists():
    # if already created, just load it
    print("Loading from local vector store")
    vector = FAISS.load_local(
        str(vec_store_path), embeddings, allow_dangerous_deserialization=True
    )
else:
    # create & save it
    print("Creating new vector store from URL...")
    text_splitter = RecursiveCharacterTextSplitter()
    documents = text_splitter.split_documents(docs)
    vector = FAISS.from_documents(documents, embeddings)
    vector.save_local(str(vec_store_path))
    print(f"Vector store saved to {str(vec_store_path)}")

Creating new vector store from URL...
Vector store saved to /home/mjbhobe/code/git-projects/LangChain/tutorial/vecstore/faiss_index_gemini


In [15]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt_wc = ChatPromptTemplate.from_template(
    """Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}"""
)

document_chain = create_stuff_documents_chain(llm, prompt_wc)

# create a retrieval chain to get the answer
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [16]:
# modify the question below & try
wc_question = """
How many groups were the participating countries divided into. In which group did India play 
and who were the other countries in that group?
"""
response = retrieval_chain.invoke({"input": wc_question})
print(response["answer"])

/home/mjbhobe/code/git-projects/LangChain/myenv/lib/python3.9/site-packages/langchain_google_genai/chat_models.py:352: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


The participating countries were divided into two groups. India played in Group A, along with Australia, New Zealand, and Zimbabwe.


In [18]:
wc_question2 = """
With which countries did England play in the Group B matches? What were the results of all 
those matches?
"""
response = retrieval_chain.invoke({"input": wc_question2})
print(response["answer"])

/home/mjbhobe/code/git-projects/LangChain/myenv/lib/python3.9/site-packages/langchain_google_genai/chat_models.py:352: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


The provided context does not specify which countries England played in the Group B matches or the results of those matches.


In [19]:
wc_question3 = """
Did the West Indies qualify for the semi finals? Where were the semi-finals and finals played. 
Who were the teams that played the semi-finals and finals? Who won the finals and by how many runs?
"""
response = retrieval_chain.invoke({"input": wc_question3})
print(response["answer"])

/home/mjbhobe/code/git-projects/LangChain/myenv/lib/python3.9/site-packages/langchain_google_genai/chat_models.py:352: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


The West Indies did not qualify for the semi finals. 

The semi-finals were played in Lahore, Pakistan and Bombay, India. The finals were played in Calcutta, India. 

The semi-finals were played between Australia and Pakistan in Lahore, Pakistan and between England and India in Bombay, India. 

The finals were played between Australia and England in Calcutta, India. Australia won the finals by 7 runs.


In [20]:
wc_question4 = """
Who were the two leading run scorers and the two leading wicket takers?
"""
response = retrieval_chain.invoke({"input": wc_question4})
print(response["answer"])

/home/mjbhobe/code/git-projects/LangChain/myenv/lib/python3.9/site-packages/langchain_google_genai/chat_models.py:352: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


The two leading run scorers were Graham Gooch (471 runs) and David Boon (447 runs). The two leading wicket takers were Craig McDermott (18 wickets) and Imran Khan (17 wickets).
